# Analítica Avanzada de Datos.
---

# Redes neuronales en PyTorch

El aprendizaje automático clásico se basa en el uso de estadísticas para determinar las relaciones entre características y etiquetas, y puede ser muy eficaz para modelos predictivos. Sin embargo, el crecimiento masivo de la disponibilidad de datos, unido a los avances en la tecnología necesarios para procesarlos, ha propiciado la aparición de nuevas técnicas de aprendizaje automático que imitan la forma en que el cerebro procesa la información en una estructura denominada red neuronal artificial

PyTorch es un marco para crear modelos de aprendizaje automático, incuidas las redes neuronales profundas (DNNs). En este notebook, utilizaremos PyTorch para crear una red neuronal simple que clasifique a los pingüinos en especies basándose en la longitud y profundidad de su culmen (pico), la longitud de sus aletas y su masa corporal

> **Cita**: El dataset de pingüinos que utilizaremos es un subconjunto de datos recogidos y puestos a disposición de la [Dr. Kristen Gorman](https://www.uaf.edu/cfos/people/faculty/detail/kristen-gorman.php)y [Palmer Station, Antarctica LTER](https://pal.lternet.edu/), miembro de [Long Term Ecological Research Network](https://lternet.edu/).

## Explorando el Dataset

Antes de empezar a utilizar PyTorch para crear el modelo, vamos a cargar los datos que necesitamos del dataset de pingüinos de las Islas Palmer, que contiene observaciones de tres especies diferentes de pingüinos.

> **Nota**: En realidad, puedes resolver el problema de clasificación de pingüinos fácilmente utilizando técnicas clásicas de aprendizaje automático sin necesidad de un modelo de aprendizaje profundo; pero es un conjunto de datos útil y facil de entender con el que podemos ejemplificar los principios de las redes neuronales.

In [ ]:
import pandas as pd

# cargar el conjunto de datos de entrenamiento (excluyendo las filas con valores nulos)
penguins = pd.read_csv('data/penguins.csv').dropna()

# Los modelos de deep learning funcionan mejor cuando las características están en escalas similares
# En una solución real, implementaríamos alguna normalización personalizada para cada característica, pero para mantener las 
# cosas simples, sólo cambiaremos la escala de FlipperLength y BodyMass para que estén en una escala similar a las medidas de la factura
penguins['FlipperLength'] = penguins['FlipperLength']/10
penguins['BodyMass'] = penguins['BodyMass']/100

# El conjunto de datos es demasiado pequeño para ser útil para el aprendizaje profundo
# Así que vamos a sobremuestrearlo para aumentar su tamaño
for i in range(1,3):
    penguins = penguins.append(penguins)

# Mostrar una muestra aleatoria de 10 observaciones
sample = penguins.sample(10)
sample

La columna **Species** es la etiqueta que predicirá nuestro modelo. Cada valor de la etiqueta representa una clase de especies de pingüinos, codificada como 0, 1 ó 2. El siguiente código muestra las especies reales a las que corresponden estas etiquetas de clase.

In [ ]:
penguin_classes = ['Amelie', 'Gentoo', 'Chinstrap']
print(sample.columns[0:5].values, 'SpeciesName')
for index, row in penguins.sample(10).iterrows():
    print('[',row[0], row[1], row[2],row[3], int(row[4]), ']',penguin_classes[int(row[-1])])

Como es habitual en un problema de aprendizaje supervisado, dividiremos el conjunto de datos en un conjunto de registros con los que entrenaremos el modelo y un conjunto más pequeño con el que validaremos el modelo entrenado

In [ ]:
from sklearn.model_selection import train_test_split

features = ['CulmenLength','CulmenDepth','FlipperLength','BodyMass']
label = 'Species'
   
# Dividimos los datos 70%-30% en los conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(penguins[features].values,
                                                    penguins[label].values,
                                                    test_size=0.30,
                                                    random_state=0)

print ('Training Set: %d, Test Set: %d \n' % (len(x_train), len(x_test)))
print("Sample of features and labels:")

# Daremos un vistazo a las primeras 25 características de entrenamiento y las etiquetas correspondientes
for n in range(0,24):
    print(x_train[n], y_train[n], '(' + penguin_classes[y_train[n]] + ')')

Las *características* son las medidas de cada observación de pingüino, y la *etiqueta* es un valor numérico que indica la especie de pingüino que representa la observación (Amelie, Papúa o Barbijo).

# Instalar e importar las bibliotecas PyTorch

Dado que planeamos utilizar PyTorch para crear nuestro clasificador de pingüinos, necesitaremos ejecutar las dos celdas siguientes para instalar e importar las bibliotecas PyTorch que pretendemos utilizar. La instalación específica de PyTorch depende de tu sistema operativo y de si tu computadora dispone de unidades de procesamiento gráfico (GPU) que puedan utilizarse para el procesamiento de alto rendimiento a través de cuda. Encontrará instrucciones detalladas en https://pytorch.org/get-started/locally/.

In [ ]:
!pip install torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as td

# Set random seed for reproducability
torch.manual_seed(0)

print("Libraries imported - ready to use PyTorch", torch.__version__)

### Preparar los datos para PyTorch

PyTorch utiliza cargadores de datos para cargar los datos de entrenamiento y validación por lotes. Ya hemos cargado los datos en arrays numpy, pero necesitamos envolverlos en conjuntos de datos PyTorch (en los que los datos se convierten en objetos tensor PyTorch) y crear cargadores para leer lotes de esos conjuntos de datos.

In [ ]:
# Crear un conjunto de datos y un cargador para los datos de entrenamiento y etiquetas
train_x = torch.Tensor(x_train).float()
train_y = torch.Tensor(y_train).long()
train_ds = td.TensorDataset(train_x,train_y)
train_loader = td.DataLoader(train_ds, batch_size=20,
    shuffle=False, num_workers=1)

# Crear un conjunto de datos y un cargador para los datos de prueba y las etiquetas
test_x = torch.Tensor(x_test).float()
test_y = torch.Tensor(y_test).long()
test_ds = td.TensorDataset(test_x,test_y)
test_loader = td.DataLoader(test_ds, batch_size=20,
    shuffle=False, num_workers=1)
print('Ready to load data')

# Definir una red neuronal

Ahora estamos listos para definir nuestra red neuronal. En este caso, vamos a crear una red que consta de 3 capas totalmente conectadas:

* Una capa de entrada que recibe un valor de entrada para cada característica (en este caso, las cuatro medidas de los pingüinos) y aplica una **función de activación ReLU**.

* Una capa oculta que recibe diez entradas y aplica una función de activación ReLU.

* Una capa de salida que utiliza una **función de activación Softmax** para generar una salida para cada especie de pingüino (que representan las probabilidades de clasificación para cada una de las tres posibles especies de pingüinos). Las funciones Softmax producen un vector con valores de probabilidad que suman 1.

In [ ]:
# Número de nodos en la capa oculta
hl = 10

# Definimos la red neuronal
class PenguinNet(nn.Module):
    def __init__(self):
        super(PenguinNet, self).__init__()
        self.fc1 = nn.Linear(len(features), hl)
        self.fc2 = nn.Linear(hl, hl)
        self.fc3 = nn.Linear(hl, len(penguin_classes))

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x),dim=1)
        return x

# Crear una instancia del modelo a partir de la red
model = PenguinNet()
print(model)

# Entrenar el modelo

Para entrenar el modelo, necesitamos alimentar repetidamente los valores de entrenamiento a través de la red, utilizar una función de pérdida (loss cost) para calcular la pérdida, utilizar un optimizador para retropropagar (backpropagation) los ajustes de peso y valor de sesgo, y validar el modelo utilizando los datos de prueba que retuvimos.

Para ello, vamos a crear una función para entrenar y optimizar el modelo, y una función para probar el modelo. A continuación, llamaremos a estas funciones de forma iterativa durante 50 épocas, el registro de la pérdida y las estadísticas de precisión para cada época.

In [ ]:
def train(model, data_loader, optimizer):
    # Poner el modelo en modo entrenamiento
    model.train()
    train_loss = 0
    
    for batch, tensor in enumerate(data_loader):
        data, target = tensor
        #feedforward
        optimizer.zero_grad()
        out = model(data)
        loss = loss_criteria(out, target)
        train_loss += loss.item()

        # retropropagar
        loss.backward()
        optimizer.step()

    #Perdida media por retorno
    avg_loss = train_loss / (batch+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss
           
            
def test(model, data_loader):
    # Cambiar el modelo en modo evaluación (para no retropropagar)
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        batch_count = 0
        for batch, tensor in enumerate(data_loader):
            batch_count += 1
            data, target = tensor
            # Obtener las predicciones
            out = model(data)

            # calcular la pérdida
            test_loss += loss_criteria(out, target).item()

            # Calcular el accuracy
            _, predicted = torch.max(out.data, 1)
            correct += torch.sum(target==predicted).item()
            
    # Calcular la pérdida media y precisión total de esta época
    avg_loss = test_loss/batch_count
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
    
    # Pérdida media de retorno para la época
    return avg_loss

# Especificación de los criterios de pérdida (CrossEntropyLoss para la clasificación multiclase)
loss_criteria = nn.CrossEntropyLoss()

# Utilizar un optimizador "Adam" par ajustar los pesos
# (consultar https://pytorch.org/docs/stable/optim.html#algorithms para detalles del algoritmo)
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.zero_grad()

# Seguiremos las métricas de cada época en estas matrices
epoch_nums = []
training_loss = []
validation_loss = []

# Entrenar más de 50 épocas
epochs = 50
for epoch in range(1, epochs + 1):

    # imprimir el número de época
    print('Epoch: {}'.format(epoch))
    
    # Introducir datos de entrenamiento en el modelo para optimizar las ponderaciones
    train_loss = train(model, train_loader, optimizer)
    
    # Introducir los datos de prueba en el modelo para comprobar su rendimiento.
    test_loss = test(model, test_loader)
    
    # Registrar las métricas de esta época
    epoch_nums.append(epoch)
    training_loss.append(train_loss)
    validation_loss.append(test_loss)

### Mientras se ejecuta el proceso de formación, entendamos que esta ocurriendo:

1. En cada época, el conjunto completo de datos de entrenamiento se transmite a través de la red. Hay cuatro características para cada observación y cuatro nodos correspondientes en la capa de entrada, por lo que las características de cada observación se transmiten como un vector de cuatro valores a esa capa. Sin embargo, por razones de eficacia, los vectores de características se agrupan en lotes, de modo que cada vez se introduce una matriz de múltiples vectores de características.

2. La matriz de valores de características es procesada por una función que realiza una *suma ponderada* utilizando pesos y *valores de sesgo* inicializados. El resultado de esta función es procesado por la **función de activación** de la capa de entrada para limitar los valores transmitidos a los nodos de la capa siguiente.

3. La *suma ponderada* y *las funciones de activación* se repiten en cada capa. Observe que las funciones operan sobre vectores y matrices en lugar de sobre valores escalares individuales. En otras palabras, el paso hacia delante es esencialmente una serie de funciones de álgebra lineal anidadas. Esta es la razón por la que los científicos de datos prefieren utilizar computadoras con unidades de procesamiento gráfico (GPU), ya que están optimizados para cálculos de matrices y vectores.

4. En la última capa de la red, los vectores de salida contienen un valor de probabilidad para cada clase posible (en este caso, las clases 0, 1 y 2). Este vector es procesado por una *función de pérdida* para determinar la distancia entre los valores calculados por la red y los valores reales. Supongamos, por ejemplo, que la observación de un pingüino papúa (clase 1) es \[0,3, 0,4, 0,3\]. La predicción correcta debería ser \[0,0, 1,0, 0,0\], por lo que la varianza entre los valores predichos y reales (lo lejos que está cada valor predicho de lo que debería ser) es \[0,3, 0,6, 0,3\]. Esta varianza se agrega para cada lote y se mantiene como un agregado en ejecución para calcular el nivel general de error (*pérdida*) incurrido por los datos de entrenamiento para la época. 

5. Al final de cada época, los datos de validación se pasan por la red y también se calculan su *pérdida* y *precisión* (proporción de predicciones correctas basadas en el valor de probabilidad más alto del vector de salida). Es importante hacer esto porque nos permite comparar el rendimiento del modelo utilizando datos en los que no se ha entrenado, lo que nos ayuda a determinar si generalizará bien para los nuevos datos o si está *sobreajustado* a los datos de entrenamiento.

6. Una vez que todos los datos han pasado a través de la red, la salida de la función de pérdida para los datos de *entrenamiento* (pero <u>no</u> los datos de *validación*) se pasa al optimizador. Los detalles precisos de cómo el optimizador procesa la pérdida varían dependiendo del algoritmo de optimización específico que se utilice; pero fundamentalmente se puede pensar en toda la red, desde la capa de entrada hasta la función de pérdida como una gran función anidada (*compuesta*). El optimizador aplica algo de **cálculo diferencial** para calcular *derivadas parciales* de la función con respecto a cada peso y valor de sesgo que se utilizó en la red. Es posible hacer esto eficientemente para una función anidada debido a algo llamado *regla de la cadena*, que permite determinar la derivada de una función compuesta a partir de las derivadas de su función interna y funciones externas. No es necesario preocuparse por los detalles matemáticos (el optimizador lo hace por ti), pero el resultado final es que las derivadas parciales nos indican la pendiente (o *gradiente*) de la función de pérdida con respecto a cada valor de peso y sesgo; en otras palabras, podemos determinar si aumentar o disminuir los valores de peso y sesgo para reducir la pérdida.

7. Una vez determinada la dirección en la que deben ajustarse los pesos y los sesgos, el optimizador utiliza la *velocidad de aprendizaje* para determinar cuánto debe ajustarlos y, a continuación, trabaja hacia atrás a través de la red en un proceso denominado *backpropagation* para asignar nuevos valores a los pesos y sesgos de cada capa.

8. Ahora la siguiente época repite todo el proceso de entrenamiento, validación y retropropagación comenzando con los pesos y sesgos revisados de la época anterior, lo que esperamos que resulte en un menor nivel de pérdida.

9. El proceso continúa así durante 50 épocas.

### Revisar las pérdidas del entrenamiento y la validación

Una vez completado el entrenamiento, podemos examinar las métricas de pérdida que registramos durante el entrenamiento y la validación del modelo. Realmente estamos buscando dos cosas:

* La pérdida debe reducirse con cada epoch, mostrando que el modelo está aprendiendo los pesos y sesgos correctos para predecir las etiquetas correctas.
* La pérdida de entrenamiento y la pérdida de validación deberían seguir una tendencia similar, mostrando que el modelo no se está sobreajustando a los datos de entrenamiento.

Vamos a trazar las métricas de pérdida y ver:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

### Ver los pesos y sesgos aprendidos
El modelo entrenado consiste en los pesos y sesgos finales que fueron determinados por el optimizador durante el entrenamiento. Basándonos en nuestro modelo de red, deberíamos esperar los siguientes valores para cada capa:

* **Capa 1:** Hay cuatro valores de entrada que van a diez nodos de salida, por lo que debería haber 10 x 4 pesos y 10 valores de sesgo.
* **Capa 2:** Hay diez valores de entrada que van a diez nodos de salida, por lo que debería haber 10 x 10 pesos y 10 valores de sesgo.
* **Capa 3:** Hay diez valores de entrada que van a tres nodos de salida, por lo que debería haber 3 x 10 pesos y 3 valores de sesgo.

In [ ]:
for param_tensor in model.state_dict():
    print(param_tensor, "\n", model.state_dict()[param_tensor].numpy())

### Evaluar el rendimiento del modelo

¿Es bueno el modelo? La precisión bruta de los datos de validación parece indicar que predice bastante bien, pero suele ser útil profundizar un poco más y comparar las predicciones de cada clase posible. Una forma habitual de visualizar el rendimiento de un modelo de clasificación es crear una matriz de confusión que muestre una tabla cruzada de predicciones correctas e incorrectas para cada clase.

In [ ]:
#Pytorch no tiene una métrica de matriz de confusión incorporada, así que usaremos SciKit-Learn
from sklearn.metrics import confusion_matrix
import numpy as np

# Poner el modelo en modo evaluación
model.eval()

# Obtener predicciones para los datos de prueba
x = torch.Tensor(x_test).float()
_, predicted = torch.max(model(x).data, 1)

# Trazar la matriz de confusión
cm = confusion_matrix(y_test, predicted.numpy())
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(penguin_classes))
plt.xticks(tick_marks, penguin_classes, rotation=45)
plt.yticks(tick_marks, penguin_classes)
plt.xlabel("Actual Species")
plt.ylabel("Predicted Species")
plt.show()

La matriz de confusión debe mostrar una línea diagonal fuerte que indique que hay más predicciones correctas que incorrectas para cada clase.

### Guardar el modelo entrenado

Ahora que tenemos un modelo que creemos razonablemente preciso, podemos guardar sus pesos entrenados para utilizarlos más adelante.

In [ ]:
# Guardar los pesos del modelo
model_file = 'models/penguin_classifier.pt'
torch.save(model.state_dict(), model_file)
del model
print('model saved as', model_file)

### Utilizar el modelo entrenado

Cuando tengamos una nueva observación de un pingüino, podemos utilizar el modelo para predecir la especie.

In [ ]:
#  Nuevas características del pingüino
x_new = [[50.4,15.3,20,50]]
print ('New sample: {}'.format(x_new))

# Crear una nueva clase de modelo y cargar los pesos
model = PenguinNet()
model.load_state_dict(torch.load(model_file))

# Poner el modelo en modo evaluación
model.eval()

# Obtener una predicción para la nueva muestra de datos
x = torch.Tensor(x_new).float()
_, predicted = torch.max(model(x).data, 1)

print('Prediction:',penguin_classes[predicted.item()])